In [1]:
from aocd import get_data
import numpy as np

- Create a 2D grid representing the map
- Identify the guard's starting position and initial facing direction (`^`, `>`, `v`, or `<`)
- If the guard faces an obstacle (`#`) in the direction they are heading, turn 90 degrees to the right
  -  Otherwise, move one step forward in the current facing direction
- Track and store all distinct positions visited by the guard, including the starting position
- Continue moving according to the rules until the guard moves outside the bounds of the grid
- Count the number of distinct positions the guard visited
- Return the total count of distinct positions

In [2]:
direction_map = {
        '^': (-1, 0),
        '>': (0, 1),
        'v': (1, 0),
        '<': (0, -1),
    }

In [3]:
def turn_right(direction):
    """New direction after turning right"""
    turns = {'^': '>', '>': 'v', 'v': '<', '<': '^'}
    return turns[direction]

In [4]:
def valid_position(grid, pos):
    """check the position is within the bounds and not an obstacle (#)"""
    rows, cols = grid.shape
    r, c = pos
    return 0 <= r < rows and 0 <= c < cols and grid[r, c] != '#'


In [56]:
def simulate_patrol(grid, start_pos, start_dir):
    rows, cols = grid.shape
    pos = start_pos
    direction = start_dir
    visited = set()
    # add starting position
    visited.add(pos)
    
    while True:
        delta_r, delta_c = direction_map[direction]
        next_pos = (pos[0] + delta_r, pos[1] + delta_c)

        if valid_position(grid, next_pos):
            pos = next_pos
            visited.add(pos)
        
        else:
            # turn right if blocked
            direction = turn_right(direction)
        
        # if the guard moves out of bounds
        if not 0 <= next_pos[0] < rows and 0 <= next_pos[1] < cols:
            break
    
    return visited

In [57]:
ex_grid_str = """
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
"""

In [52]:
example_grid = np.array([list(line) for line in ex_grid_str.strip().split("\n")]); grid

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U1')

In [53]:
for i, row in enumerate(example_grid):
    for j, cell in enumerate(row):
        if cell in direction_map:
            start_pos = (i,j)
            start_dir = cell

In [54]:
visited_positions = simulate_patrol(example_grid, start_pos, start_dir)

In [55]:
len(visited_positions)

41

Now for the real data:

In [63]:
grid = np.array([list(line) for line in get_data(day=6, year=2024).strip().split("\n")])

In [64]:
for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        if cell in direction_map:
            start_pos = (i,j)
            start_dir = cell

In [65]:
visited_positions = simulate_patrol(grid, start_pos, start_dir)

In [66]:
len(visited_positions)

4656

### Part Two

- Follow the guard's movement rules (move forward unless blocked, then turn right) until they exit the grid or repeat a position and direction
- Keep track of every (position, direction) pair the guard visits
- If the guard visits the same position and direction more than once, they are in a loop
- For each empty cell (`.`) not currently visited, temporarily place an obstruction (`#`) and simulate the guard's patrol again
- Check if the guard gets stuck in a loop with the obstruction in place
- Count the positions where adding an obstruction causes the guard to enter a loop
- Exclude the guard's starting position
